In [1]:
import json
import pandas as pd

In [2]:
conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/HiVerilog_expansion_eval/without_head/conversations.json'
graph_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/HiVerilog_expansion_eval/without_head/graph.jsonl'

In [3]:
import re

def find_module_string(text):
    # Regex pattern to find the string starting with "module" and ending with ";"
    # It uses non-greedy matching to find the shortest match
    pattern = r"module.*?;"
    
    # Using re.DOTALL to make the dot match all characters including newline
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group()  # Returns the matched string
    else:
        return None  # No match found

# Example usage
input_text = """
some initial text
module name="example1" \n   attribu\nte="   value";
some other text
module name="example2" attribute="another value";
"""

# Find the first occurrence
result = find_module_string(input_text)
print("Found substring:", result)

# If you need to find all occurrences instead of just the first one, you can use re.findall
all_matches = re.findall(r"module.*?;", input_text, re.DOTALL)
print("All found substrings:", all_matches)

Found substring: module name="example1" 
   attribu
te="   value";
All found substrings: ['module name="example1" \n   attribu\nte="   value";', 'module name="example2" attribute="another value";']


In [4]:
graph = pd.read_json(graph_path, lines=True)
with open(conv_path, 'r') as f:
    conv = json.load(f)


In [5]:
conv[0]['conversations'][0]['value']

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'clk', 'type': 'input port'}, {'id': 1, 'content': 'rst', 'type': 'input port'}, {'id': 2, 'content': 'mode', 'type': 'input port'}, {'id': 3, 'content': 'op1', 'type': 'input port'}, {'id': 4, 'content': 'op2', 'type': 'input port'}, {'id': 5, 'content': 'result', 'type': 'output port'}, {'id': 6, 'content': 'u_adder', 'type': 'submodule'}, {'id': 7, 'content': 'u_subtractor', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[4, 3, 4, 3, 2, 6, 7], [6, 6, 7, 7, 5, 5, 5]]\n, Implement a modular 32-bit Adder-Subtractor using Verilog. In the top module named `as_module`, instantiate submodules for addition and subtraction. The `as_module` orchestrates these submodules to perform either addition or subtraction based on a mode input.\n\nModule name:\n    as_module\n\nInput ports:\n    clk: Clock signal for synchronous operations.\n    rst: Reset signal, high to reset, low otherwise.\n    mode: Op

In [6]:
conv[0]['conversations'][1]['value']

'module as_module(\n    input clk,\n    input rst,\n    input mode,\n    input [31:0] op1,\n    input [31:0] op2,\n    output [31:0] result\n);\n\nwire [31:0] add_result;\nwire [31:0] sub_result;\n\n// Instantiating the adder module\nadder u_adder (\n    .a(op1),\n    .b(op2),\n    .sum(add_result)\n);\n\n// Instantiating the subtractor module\nsubtractor u_subtractor (\n    .a(op1),\n    .b(op2),\n    .difference(sub_result)\n);\n\n// Selecting output based on mode\nassign result = (mode == 0) ? add_result : sub_result;\n\nendmodule\n\n// Adder module\nmodule adder(\n    input [31:0] a,\n    input [31:0] b,\n    output [31:0] sum\n);\n    assign sum = a + b;\nendmodule\n\n// Subtractor module\nmodule subtractor(\n    input [31:0] a,\n    input [31:0] b,\n    output [31:0] difference\n);\n    assign difference = a - b;\nendmodule\n'

In [7]:
head = find_module_string(conv[0]['conversations'][1]['value'])
conv[0]['conversations'][0]['value'] + head

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'clk', 'type': 'input port'}, {'id': 1, 'content': 'rst', 'type': 'input port'}, {'id': 2, 'content': 'mode', 'type': 'input port'}, {'id': 3, 'content': 'op1', 'type': 'input port'}, {'id': 4, 'content': 'op2', 'type': 'input port'}, {'id': 5, 'content': 'result', 'type': 'output port'}, {'id': 6, 'content': 'u_adder', 'type': 'submodule'}, {'id': 7, 'content': 'u_subtractor', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[4, 3, 4, 3, 2, 6, 7], [6, 6, 7, 7, 5, 5, 5]]\n, Implement a modular 32-bit Adder-Subtractor using Verilog. In the top module named `as_module`, instantiate submodules for addition and subtraction. The `as_module` orchestrates these submodules to perform either addition or subtraction based on a mode input.\n\nModule name:\n    as_module\n\nInput ports:\n    clk: Clock signal for synchronous operations.\n    rst: Reset signal, high to reset, low otherwise.\n    mode: Op

In [8]:
for i in range(len(conv)):
    module_head = find_module_string(conv[i]['conversations'][1]['value'])
    conv[i]['conversations'][0]['value'] = conv[i]['conversations'][0]['value'] + '\n' + module_head

In [ ]:
# conv[100]['conversations'][0]['value']

IndexError: list index out of range

In [10]:
new_conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/HiVerilog_expansion_eval/with_head/conversations.json'



In [11]:
with open(new_conv_path, 'w') as f:
    json.dump(conv, f)

In [12]:
import json
with open(new_conv_path, 'r') as f:
    new_conv = json.load(f)
    


In [13]:
len(new_conv)

480

In [ ]:
new_conv[100]['conversations'][0]['value']

In [32]:
len(graph)

2148